In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import torchvision.models as models
import os
import torch.optim as optim
from torch.autograd import Variable
import glob
import skimage.color as sc


import os 
#from google.colab import drive
#drive.mount('/content/gdrive')
#os.chdir('/content/gdrive/My Drive/SRGAN')

from loss import GeneratorLoss
from dataload.srdataload import srDataset
from utils.util import psnr , PSNR , predict_simage


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
learning_rate = 6.3e-4  #6.3e-4 1e-5

class residualBlock(nn.Module):
    def __init__(self):
        super(residualBlock,self).__init__()
        self.channels = 3
        self.num_filter = 64
        self.conv1= nn.Conv2d (self.num_filter,self.num_filter,(3,3) ,stride=1,padding = 'same')
        self.batchn1 = nn.BatchNorm2d (self.num_filter)
        self.prelu1 = nn.PReLU()
        self.conv2= nn.Conv2d (self.num_filter,self.num_filter,(3,3) ,stride=1,padding = 'same')
        self.batchn2 = nn.BatchNorm2d (self.num_filter)
        
        
    def forward(self,x):
        in_x = x
        out = self.conv1(x)
        out = self.batchn1(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.batchn2(out)
        out = out + in_x
        
        return out
    
class  Generator(nn.Module):
    
    def __init__(self):
        super(Generator,self).__init__()
        self.channels = 3
        self.num_filter = 64
        self.conv1= nn.Conv2d (self.channels,self.num_filter,(9,9) ,stride=1,padding=4)
        self.prelu1 = nn.PReLU()
        self.f1 = self.f_layers(residualBlock,5)
        
        self.conv2= nn.Conv2d (self.num_filter,self.num_filter,(3,3) ,stride=1,padding =1)
        self.batchn2 = nn.BatchNorm2d (self.num_filter)
        
        # 卷积 后放大图片
        self.conv3 = nn.Conv2d (self.num_filter,4 * self.num_filter,(3,3) ,stride=1,padding=1)
        self.pshuffle1 = nn.PixelShuffle(2)
        self.prelu3 = nn.PReLU()
        
        self.conv4 = nn.Conv2d ( self.num_filter,4 * self.num_filter,(3,3) ,stride=1,padding=1)
        self.pshuffle2 = nn.PixelShuffle(2)
        self.prelu4 = nn.PReLU()
        
        self.conv5= nn.Conv2d (self.num_filter,self.channels,(9,9) ,stride=1,padding=4)
        
        
        
       
    
    def f_layers ( self,block,num):
        layers =[]
        for i in range(num):
            layers.append(block())
            
        return nn.Sequential ( *layers )
            
        
        
    def forward(self,x):
        
        out = self.conv1(x)
        out = self.prelu1(out)
        in_out = out
        out = self.f1(out)
        out = self.conv2(out)
        out = self.batchn2(out)
        out = in_out + out
        
        out = self.conv3(out)
        out = self.pshuffle1(out)
        out = self.prelu3(out)
        
        out = self.conv4(out)
        out = self.pshuffle2(out)
        out = self.prelu4(out)
        
        out = self.conv5(out)
        
        return (torch.tanh(out) + 1)/2
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.channels = 3
        self.num_filter = 64
        self.conv1= nn.Conv2d (self.channels,self.num_filter,(3,3) ,stride=1,padding=1)
        self.lrelu1 = nn.LeakyReLU(0.2)
    
        self.conv2= nn.Conv2d (self.num_filter,self.num_filter,(3,3) ,stride=2,padding=1)
        self.batchn2 = nn.BatchNorm2d (self.num_filter)
        self.lrelu2 = nn.LeakyReLU(0.2)
       
        self.conv3= nn.Conv2d (self.num_filter,2 * self.num_filter,(3,3) ,stride=1,padding=1)
        self.batchn3 = nn.BatchNorm2d ( 2 * self.num_filter)
        self.lrelu3 = nn.LeakyReLU(0.2)
        self.conv4= nn.Conv2d ( 2 * self.num_filter,2 * self.num_filter,(3,3) ,stride=2,padding=1)
        self.batchn4 = nn.BatchNorm2d ( 2 * self.num_filter)
        self.lrelu4 = nn.LeakyReLU(0.2)
        
        self.conv5= nn.Conv2d (2 * self.num_filter,4 * self.num_filter,(3,3) ,stride=1,padding=1)
        self.batchn5 = nn.BatchNorm2d ( 4 * self.num_filter)
        self.lrelu5 = nn.LeakyReLU(0.2)
        self.conv6= nn.Conv2d ( 4 * self.num_filter,4 * self.num_filter,(3,3) ,stride=2,padding=1)
        self.batchn6 = nn.BatchNorm2d ( 4 * self.num_filter)
        self.lrelu6 = nn.LeakyReLU(0.2)
        
        self.conv7= nn.Conv2d (4 * self.num_filter,8 * self.num_filter,(3,3) ,stride=1,padding=1)
        self.batchn7 = nn.BatchNorm2d ( 8 * self.num_filter)
        self.lrelu7 = nn.LeakyReLU(0.2)
        self.conv8= nn.Conv2d ( 8 * self.num_filter,8 * self.num_filter,(3,3) ,stride=2,padding=1)
        self.batchn8 = nn.BatchNorm2d ( 8 * self.num_filter)
        self.lrelu8 = nn.LeakyReLU(0.2)
        
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(512, 1024, kernel_size=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(1024, 1, kernel_size=1)
           
            
        )
        
        
    def forward(self,x):
        batch_size = x.size(0)
        out = self.conv1(x)
        out = self.lrelu1(out)
        
        out = self.conv2(out)
        out = self.batchn2(out)
        out = self.lrelu2(out)
        
        out = self.conv3(out)
        out = self.batchn3(out)
        out = self.lrelu3(out)
        out = self.conv4(out)
        out = self.batchn4(out)
        out = self.lrelu4(out)
        
        out = self.conv5(out)
        out = self.batchn5(out)
        out = self.lrelu5(out)
        out = self.conv6(out)
        out = self.batchn6(out)
        out = self.lrelu6(out)
        
        out = self.conv7(out)
        out = self.batchn7(out)
        out = self.lrelu7(out)
        out = self.conv8(out)
        out = self.batchn8(out)
        out = self.lrelu8(out)
           
        out = self.fc(out).view(batch_size)
        
        return torch.sigmoid(out)
        

def train_model( g_model,d_model,dataset1,steps,batch_size ,loss1,device,learning_rate ):
    train_dataloader = DataLoader(dataset1, batch_size= batch_size, shuffle=True)
    
    loss_f = loss1
    size = len(train_dataloader.dataset)
    
    optimizerG = optim.Adam(g_model.parameters(),learning_rate)
    optimizerD = optim.Adam(d_model.parameters(),learning_rate)
  
    img_path = 'test_imgs/b_4down3.jpg'
    img_path2 = 'test_imgs/butterfly.png'
    #img_path = 'test_imgs/down1.jpg'
    #img_path2 = 'test_imgs/t2_1.png'
    data_transform = transforms.Compose([
    transforms.ToTensor()#,transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    
    for step in range( steps):
        g_model.train()
        d_model.train()
        for batch , (X, y) in enumerate(train_dataloader):
        # Compute prediction and loss
           
            input1 ,target = Variable(X), Variable(y, requires_grad=False)
            input1 = input1.to(device)
            target = target.to(device)
           
            real_img =target
            fake_img = g_model(input1)
             # 更新判别器D的参数
            d_model.zero_grad()
            real_out = d_model(real_img).mean()
            fake_out = d_model(fake_img).mean()
            d_loss = 1 - real_out + fake_out
            d_loss.backward(retain_graph=True)
            optimizerD.step()
            
            # 更新生成器G的参数
            g_model.zero_grad()
            
            fake_img = g_model(input1)
            fake_out = d_model(fake_img).mean()
            ##
            #print ( fake_out.shape)
            #print ( fake_img.shape)
            #print ( real_img.shape)
            g_loss = generator_criterion(fake_out, fake_img, real_img)
            g_loss.backward()
            
            fake_img = g_model(input1)
            fake_out = d_model(fake_img).mean()
            
            
            optimizerG.step()
       
            predict_simage ( g_model,img_path,img_path2,data_transform,device)

            if batch % 100 == 0:
                loss, current = g_loss.item(), batch * len(X)
                predict_simage ( g_model,img_path,img_path2,data_transform,device)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

            ''' if (batch+1) % 500 == 0:
              torch.save(g_model.state_dict(), 'save_models/'+str(step)+'_'+str(batch)+'gmodel_weights2.pth')
              torch.save(d_model.state_dict(), 'save_models/'+str(step)+'_'+str(batch)+'dmodel_weights2.pth')
            if (batch+1) % 700 == 0:
              torch.save(g_model.state_dict(), 'save_models/'+str(step)+'_'+str(batch)+'gmodel_weights2.pth')
              torch.save(d_model.state_dict(), 'save_models/'+str(step)+'_'+str(batch)+'dmodel_weights2.pth')
            
            if (batch+1) % 1000 == 0:
              torch.save(g_model.state_dict(), 'save_models/'+str(step)+'_'+str(batch)+'gmodel_weights2.pth')
              torch.save(d_model.state_dict(), 'save_models/'+str(step)+'_'+str(batch)+'dmodel_weights2.pth')'''


        torch.save(g_model.state_dict(), 'save_models/'+str(step)+'gmodel_weights2.pth')
        torch.save(d_model.state_dict(), 'save_models/'+str(step)+'dmodel_weights2.pth')
        predict_simage ( g_model,img_path,img_path2,data_transform,device)

if __name__ == "__main__":
    
    #hr_path = r'gan_data/hr_hdf5_file.h5'
    #lr_path = r'gan_data/lr_hdf5_file.h5'
    hr_path = r'D:/xiao mai/SR_datasets/SR/processed/gan_data/hr_hdf5_file.h5'
    lr_path =  r'D:/xiao mai/SR_datasets/SR/processed/gan_data/lr_hdf5_file.h5'

    data_transform = transforms.Compose([
        transforms.ToTensor()#,transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    sr_data = srDataset( lr_dir = lr_path,hr_dir = hr_path, mode='train', transform=data_transform)
    generator = Generator().to(device)
    print('# generator parameters:', sum(param.numel() for param in generator.parameters()))
    discriminator = Discriminator().to(device) #Discriminator
    print('# discriminator parameters:', sum(param.numel() for param in discriminator.parameters()))
    #C:\Users\maijieai\pratice\p1\torch_files\SRGAN-20220509T100244Z-002\SRGAN\save_models
   # generator.load_state_dict(torch.load('C:/Users/maijieai/pratice/p1/torch_files/SRGAN-c/SRGAN/save_models/9gmodel_weights2.pth'))
    #discriminator.load_state_dict(torch.load('save_models/9dmodel_weights2.pth'))
    generator_criterion = GeneratorLoss().to(device)
    #print(sr_data[5][0].shape)  # 此处为__getitem__的用法
    
    train_model( generator,discriminator,sr_data,20,64 ,generator_criterion,device,learning_rate)
        

Using cuda device
# generator parameters: 734219
# discriminator parameters: 5215425
11.415642015529421
11.415642015529423
11.415642015529421
11.415642015529423
loss: 0.074648  [    0/67902]
12.330867382581467
12.330867382581465
11.30675822601848
11.306758226018484
11.363680459110967
11.36368045911097
12.329983395735006
12.329983395735006
12.61100779552141
12.61100779552141
12.517484837377445
12.517484837377443
12.404872447426051
12.404872447426055
12.304731215956828
12.304731215956831
12.555710736338064
12.555710736338064
12.832471644095257
12.83247164409526
12.466635570347862
12.466635570347865
12.784170298439903
12.784170298439907
12.97933261395957
12.97933261395957
13.121496568442502
13.121496568442508
13.155328418034514
13.155328418034518
12.783379724679165
12.783379724679167
12.768474319344605
12.768474319344612
13.169224038748212
13.169224038748213
13.330341328106485
13.330341328106485
13.521967533700533
13.52196753370053
13.41405717543595
13.41405717543595
13.307874294782039
13

16.137592555714143
16.137592555714143
15.96475063735997
15.964750637359973
15.803950388068202
15.803950388068206
15.852227681303972
15.852227681303974
16.127787489356784
16.12778748935678
16.135882940222565
16.13588294022257
16.024826167906625
16.024826167906628
15.72709113126551
15.727091131265514
15.640478285308362
15.640478285308365
15.468038368182974
15.468038368182974
15.423726733061933
15.423726733061933
15.474004396324538
15.474004396324542
15.467914605725113
15.467914605725115
15.624033740272495
15.624033740272496
15.84846315185164
15.848463151851638
16.155923721916658
16.155923721916658
16.35543966614732
16.35543966614732
16.528616227747257
16.52861622774726
16.469640744647574
16.469640744647574
16.298770250104322
16.298770250104322
16.496999923433556
16.49699992343356
16.344904511300548
16.344904511300548
15.983361849823922
15.98336184982392
15.761395897493017
15.761395897493015
15.79947643460499
15.79947643460499
15.88585140287526
15.885851402875264
15.976976190944008
15.976

17.311605139596427
17.311605139596427
17.035122769284346
17.03512276928435
16.616842847795386
16.61684284779539
16.48150793057954
16.48150793057954
16.48064386712256
16.480643867122563
16.666344683602368
16.666344683602368
16.810796045294936
16.81079604529494
16.639309997916747
16.639309997916747
16.238003396583817
16.238003396583817
16.100635512185484
16.10063551218548
16.31765809651778
16.317658096517782
16.594269881377915
16.59426988137792
16.817741400285843
16.817741400285843
16.823429668630748
16.82342966863075
16.76239695797885
16.76239695797885
16.596733971425987
16.596733971425987
16.488712852233313
16.488712852233313
16.39144283016624
16.391442830166238
16.231562443907258
16.23156244390726
16.169850380044572
16.169850380044572
16.172436888045276
16.172436888045276
16.314586532672863
16.314586532672863
16.37618202740735
16.37618202740735
16.42329289723824
16.42329289723824
16.294837581760362
16.294837581760365
15.908788478813776
15.908788478813776
15.910290478235055
15.91029047

16.79181415277857
16.79181415277857
16.99823394710613
16.998233947106133
17.151984299602457
17.15198429960246
17.227617180531738
17.22761718053174
17.287679263792235
17.28767926379224
17.27940706229665
17.27940706229665
17.275413783442534
17.275413783442538
17.042374717097154
17.042374717097154
16.878667120150784
16.878667120150784
16.96094830273027
16.960948302730273
16.846662215322297
16.846662215322297
16.66965238190161
16.669652381901614
16.422672512121995
16.422672512122
16.303548734985874
16.303548734985878
16.247827096061847
16.247827096061847
16.361456836656572
16.361456836656576
16.385441814546866
16.38544181454687
16.341625761277392
16.341625761277395
16.50087419573417
16.500874195734173
16.641620308295362
16.641620308295362
16.982278656054316
16.982278656054316
17.24889530069946
17.24889530069946
17.407287107486447
17.40728710748645
17.39429301601794
17.39429301601794
17.056510164593846
17.05651016459385
16.7722874525741
16.7722874525741
16.570569771483655
16.570569771483655

16.805826015751173
16.805826015751176
16.655207595952035
16.65520759595204
16.58231402675019
16.58231402675019
16.745378722033855
16.745378722033855
16.910845984025045
16.91084598402505
17.034726096196373
17.034726096196373
17.092942560987076
17.09294256098708
17.091867333497614
17.091867333497618
16.99242193836644
16.99242193836644
17.145589023449293
17.145589023449293
17.536246029397944
17.536246029397947
17.714644458105713
17.714644458105713
17.596838063924338
17.59683806392434
17.19927208558108
17.199272085581082
16.646638990269278
16.64663899026928
16.135868205083447
16.135868205083447
15.9490232148883
15.9490232148883
16.152505837286487
16.152505837286487
16.39954405042718
16.39954405042718
16.50689981654768
16.506899816547683
16.54841991893963
16.548419918939633
16.753785103175225
16.753785103175225
16.97547407543931
16.97547407543931
17.30706244458292
17.307062444582918
17.45995104388787
17.45995104388787
17.51526604672679
17.51526604672679
17.47193061513428
17.471930615134283


...

20.029262637599572
20.02926263759957
19.96800616974683
19.968006169746833
19.975291611175848
19.975291611175848
20.02558534451412
20.025585344514116
20.124521244511048
20.12452124451105
20.08887584444812
20.088875844448122
19.99615962762296
19.99615962762296
20.09332971363667
20.093329713636674
20.128207732082195
20.128207732082192
20.08291051555232
20.082910515552324
20.005847959666575
20.00584795966658
19.897612625260443
19.897612625260443
19.885928360114054
19.885928360114054
20.033072435389958
20.033072435389954
20.100516148874718
20.100516148874718
19.977313483046377
19.97731348304638
19.816970069793243
19.816970069793246
19.856473094714556
19.85647309471456
20.09228025566854
20.09228025566854
20.09228025566854
20.09228025566854
loss: 0.007322  [12800/67902]
20.193041675795495
20.19304167579549
19.95995413042082
19.95995413042082
19.753475053515835
19.753475053515842
19.792049305911164
19.792049305911167
19.97665618020638
19.976656180206376
20.074986611037133
20.074986611037136
19

20.099411148734465
20.099411148734468
19.941979630406255
19.94197963040626
19.941979630406255
19.94197963040626
loss: 0.007036  [25600/67902]
20.140096378556983
20.140096378556983
20.30403206781487
20.304032067814873
20.267286801692325
20.26728680169233
20.037178308192978
20.03717830819298
19.913242571013576
19.91324257101358
20.087448525780932
20.087448525780935
20.335674062454963
20.335674062454963
20.344452255688868
20.34445225568887
20.10465749853096
20.104657498530962
19.881752304005925
19.881752304005925
20.10070305476372
20.10070305476372
20.313402774330893
20.313402774330896
20.233199014683652
20.233199014683656
20.066319598830532
20.066319598830535
19.974746065705002
19.974746065705006
20.052203073376173
20.052203073376177
20.07452087575281
20.074520875752814
20.044269055636292
20.044269055636295
20.023280624645754
20.023280624645757
19.958623870506653
19.958623870506656
19.86170197401018
19.86170197401018
19.943963916334816
19.943963916334813
20.13980336464692
20.139803364646

19.772085719004135
19.772085719004135
19.48348116627497
19.48348116627497
19.133179846102408
19.133179846102408
19.281087785926452
19.281087785926452
19.563490741494252
19.563490741494252
19.62222829590283
19.622228295902833
19.50699819526225
19.506998195262252
19.402328176087675
19.402328176087675
19.42207545113779
19.42207545113779
19.63210857385122
19.63210857385122
19.629067648224613
19.629067648224613
19.574850174561373
19.574850174561373
19.517222752957984
19.517222752957988
19.58399116546205
19.583991165462045
19.760685293021712
19.760685293021712
19.81555864355539
19.81555864355539
19.671917243006995
19.671917243006995
19.609382424550123
19.609382424550123
19.78934324790394
19.789343247903943
19.815871212258514
19.815871212258518
19.79458791335793
19.79458791335793
19.6259570404262
19.625957040426204
19.689569105755467
19.689569105755467
19.925285537719464
19.925285537719468
20.03055559874791
20.030555598747913
19.850035821109866
19.850035821109866
19.749548284685545
19.7495482

19.76179381317018
19.76179381317018
19.841066990456873
19.841066990456873
19.7064173794267
19.7064173794267
19.567407002285748
19.56740700228575
19.692994155202644
19.692994155202648
19.914130138131544
19.914130138131544
19.962583137010085
19.96258313701009
19.878155749152114
19.878155749152118
19.732247017042653
19.732247017042653
19.85804859379386
19.85804859379386
20.01222539674667
20.012225396746672
20.07214196750644
20.07214196750644
19.916027766370835
19.91602776637084
19.79991054914908
19.799910549149086
19.981958797949105
19.9819587979491
20.05052833638694
20.05052833638694
19.985590382191624
19.985590382191628
19.84646857332873
19.846468573328732
19.781625528047428
19.78162552804743
19.919799099660406
19.91979909966041
20.039435705162575
20.039435705162575
20.001129328388597
20.0011293283886
19.926811246778644
19.926811246778648
19.835749050244782
19.835749050244782
19.833440091744944
19.833440091744947
19.91985232497332
19.91985232497332
19.945357320597086
19.945357320597086


19.832900700679264
19.832900700679268
19.70239777800508
19.702397778005082
19.760041454904314
19.760041454904318
19.8802530470365
19.8802530470365
19.797108553509112
19.797108553509116
19.566498991674674
19.566498991674674
19.507020029406377
19.507020029406377
19.689088273609386
19.689088273609386
19.69592435307862
19.695924353078624
19.5359614085274
19.5359614085274
19.37146120384631
19.37146120384631
19.481140069598933
19.481140069598933
19.653948723878266
19.653948723878266
19.653948723878266
19.653948723878266


(64, 64, 3)
(256, 256, 4)


In [3]:
import pytorch_ssim

def predict_simaget ( model,img_path,img_path2,transforms1,device,num):
    img = Image.open(img_path)
    target = Image.open(img_path2)
    imgt = transforms1(img).unsqueeze(0)
    imgt = imgt.to(device)
    
    imgn = np.array (img)
    target = np.array ( target)
    print ( imgn.shape)
    
    pre = model(imgt)
    pren = pre.cpu().detach().numpy().squeeze(0)
    pren *= 225
    pren = np.float32(pren).transpose(1,2,0)
    #pre_t = detransform(pre,device)
    #print ( imgn)
    #print ( pre_t)
    pre_s = pren.astype(np.uint8)
    pre_s1 = Image.fromarray ( pre_s)
    #pre_s1.save ("test_set5/normal_data/test_image"+str(num)+".png")
    print ( pren.shape)
    p1 = psnr (target,pren)
    p3 = PSNR ( pren,target)
    #p4 = PSNR (imgn,target)
    #p2 = psnr ( target,imgn )
    #sr_image = model(lr_image)
    mse = ((target - pren) ** 2).data.mean()
    psnr2 = 10 * log10(1 / mse)
    ssim = pytorch_ssim.ssim(pren, target).data[0]
    print ( pnsr2)
    print ( ssim )
    print ( p1)
    print ( p3)
    #print ( p4)
    #print ( p2)
    return ssim

cr = 1
ssi = 0
for i in range(9):
    
    s1 =0
    
    p1 = "C:/e/SR_datasets/SR/test/set14/t"+str(i+1)+".png"
    p2 = "C:/e/SR_datasets/SR/test/set14/down"+str(i+1)+".png"
    p3 = "C:/e/SR_datasets/SR/test/set14/cubic"+str(i+1)+".png"
    print (i)
    if (i==3)|(i==4)|(i==8) :
        continue
        
    s1=predict_simaget ( generator,p2,p1,data_transform,device,i+1)
    cr+=1
    ssi+=s1
    
print ( ssi/(cr*1.0))

img_path = 'test_imgs/b_4down1.jpg'
img_path2 = 'test_imgs/baby.png'
#predict_simaget ( generator,img_path,img_path2,data_transform,device,31)
img_path = 'test_imgs/b_4down2.jpg'
img_path2 = 'test_imgs/woman.png'
#predict_simaget ( generator,img_path,img_path2,data_transform,device,41)
'''
sr_image = model(lr_image)
mse = ((hr_image - sr_image) ** 2).data.mean()
psnr = 10 * log10(1 / mse)
ssim = pytorch_ssim.ssim(sr_image, hr_image).data[0]
'''



0
(120, 125, 3)
(480, 500, 3)


AttributeError: 'memoryview' object has no attribute 'mean'

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!pip install scikit-image
!pip install glob2
!pip install torch
!pip install torchtext 
!pip install torchvision
!pip install torchaudio
!pip install h5py

In [ ]:
!nvidia-smi

Tue May  3 08:40:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------